<a href="https://colab.research.google.com/github/heyoo807/laboratory/blob/master/%EB%A9%9C%EB%A1%A0_%EC%8B%9C%EB%8C%80%EB%B3%84%EC%B0%A8%ED%8A%B8_%ED%81%AC%EB%A1%A4%EB%A7%81_beautifulsoup%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [322]:
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests as req
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

In [314]:
# ajax로 받은 url 안 문자 변경을 반복해서 시대별 차트 노래의 songId 크롤링
import re

for year in range(2021, 2024):  # 1985부터 2023까지 년도 변경
    base_url = 'https://www.melon.com/chart/age/list.htm?idx={}&chartType=YE&chartGenre=KPOP&chartDate={}&moved=Y'

    albumids_name = f'albumids_{year}'
    globals()[albumids_name] = []
    for idx in [1]:  # idx는 1 또는 51
        url = base_url.format(idx, year)
        res = req.get(url, headers=header)
        html = bs(res.text)

        a_tags = html.find_all('a')
        res = []
        filtered_hrefs = []

        # 각 a 태그의 href 속성 값 출력
        for a_tag in a_tags:
            href_value = a_tag.get('href')
            res.append(href_value)
        filtered_hrefs = [i for i in res if 'playSong' in i]

        for i in range(len(filtered_hrefs)):
          match = re.search(r"javascript:melon\.play\.playSong\('\d+','(\d+)'\);", filtered_hrefs[i])
          extracted_number = match.group(1)
          globals()[albumids_name].append(extracted_number)
        print(len(globals()[albumids_name]))



100
100
100


In [336]:
for year in range(2021, 2024):
  base_url = 'https://m2.melon.com/song/lyrics.htm?songId={}'
  df_name = f'df_{year}'
  albumids_name = f'albumids_{year}'
  globals()[df_name] = pd.DataFrame(columns = ['title', 'artist', 'album', 'genre', 'date'])

  for i in range(len(globals()[albumids_name])):
    url = base_url.format(globals()[albumids_name][i])
    print(url)
    res = req.get(url, headers=header)
    html = bs(res.text)

    title = html.select('.song_name')
    artist = html.select('.artist>a>span')
    album = html.select('.list>dd:nth-child(2)')
    date = html.select('.list>dd:nth-child(4)')
    genre = html.select('.list>dd:nth-child(6)')

    for t, a, ab, g, d in zip(title, artist, album, genre, date):
      t = t.text.split('곡명')[-1].strip()
      d = d.text.split('.')[0]
      data = [t, a.text, ab.text, g.text, d]
      series_to_append = pd.Series(data,index = ['title', 'artist', 'album', 'genre', 'date']).to_frame().T
      globals()[df_name] = pd.concat([globals()[df_name],series_to_append], ignore_index = True)

      # globals()[df_name] = globals()[df_name].append(pd.Series(data,index = ['title', 'artist', 'album', 'genre', 'date']), ignore_index = True)
      print(i)

https://m2.melon.com/song/lyrics.htm?songId=33239419
0
https://m2.melon.com/song/lyrics.htm?songId=30287019
1
https://m2.melon.com/song/lyrics.htm?songId=32872978
2
https://m2.melon.com/song/lyrics.htm?songId=33487342
3
https://m2.melon.com/song/lyrics.htm?songId=33372781
4
https://m2.melon.com/song/lyrics.htm?songId=33061995
5
https://m2.melon.com/song/lyrics.htm?songId=33480898
6
https://m2.melon.com/song/lyrics.htm?songId=33507137
7
https://m2.melon.com/song/lyrics.htm?songId=3414749
8
https://m2.melon.com/song/lyrics.htm?songId=33167063
9
https://m2.melon.com/song/lyrics.htm?songId=33625988
10
https://m2.melon.com/song/lyrics.htm?songId=33013877
11
https://m2.melon.com/song/lyrics.htm?songId=33503722
12
https://m2.melon.com/song/lyrics.htm?songId=32961718
13
https://m2.melon.com/song/lyrics.htm?songId=32794652
14
https://m2.melon.com/song/lyrics.htm?songId=33077590
15
https://m2.melon.com/song/lyrics.htm?songId=32860397
16
https://m2.melon.com/song/lyrics.htm?songId=33464805
17
htt

In [337]:
with pd.ExcelWriter('output_file.xlsx') as writer:
    for year in range(2021, 2024):
        df_name = f'df_{year}'

        # 데이터프레임을 Excel 파일에 시트로 저장
        globals()[df_name].to_excel(writer, sheet_name=str(year), index=False)

### 추후 작곡가 별 데이터프레임 생성 에활용할 것

In [ ]:
    composer_comment = html.find(text=lambda text: isinstance(text, Comment) and '작사가' in text)
    next_li_elements = composer_comment.find_next_siblings('li')

    for next_li in next_li_elements:
        lyricists = []
        composers = []
        arrangers = []
        # 선택한 li 태그 안에 있는 composer-info 클래스를 가진 div 태그들 추출
        composer_divs = next_li.select('.entry')

        # composer-info 클래스를 가진 div 태그 안에 있는 글자 출력
        for composer_div in composer_divs:
            # print( composer_div.get_text(strip=True))

            line = composer_div.get_text(strip=True)

            # 뒤에 붙은 단어(작곡, 작사, 편곡) 삭제하고 역할에 따라 리스트에 추가
            role_removed = line[:-2].strip()
            role = line[-2:]

            if role == '작사':
                lyricists.append(role_removed)
                print('작사가 리스트:', lyricists)
            elif role == '작곡':
                composers.append(role_removed)
                print('작곡가 리스트:', composers)
            elif role == '편곡':
                arrangers.append(role_removed)
                print('편곡가 리스트:', arrangers)

            # 결과 출력
        print(len(lyricists))